In [3]:
#!wget --no-check-certificate \
#    https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB-Training_fixed.zip -O /tmp/GTSRB-Training_fixed.zip

--2019-12-21 23:06:35--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB-Training_fixed.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187490228 (179M) [application/zip]
Saving to: ‘/tmp/GTSRB-Training_fixed.zip’

/tmp/GTSRB-Training 100%[===================>] 178.80M  11.1MB/s    in 19s     

2019-12-21 23:06:55 (9.50 MB/s) - ‘/tmp/GTSRB-Training_fixed.zip’ saved [187490228/187490228]



The following python code will use the OS library to use Operating System libraries, giving you access to the file system, and the zipfile library allowing you to unzip the data.

In [51]:
import os
import zipfile

#local_zip = '/content/drive/My Drive/tsrd-train.zip'
#zip_ref = zipfile.ZipFile(local_zip, 'r')
#zip_ref.extractall('/tmp/chinaSign')

#zip_ref.close()

BadZipFile: ignored

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Directory with our training horse pictures
#train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
#train_human_dir = os.path.join('/tmp/horse-or-human/humans')

In [0]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [75]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 6400)             

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [80]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/tsrd-train',  # This is the source directory for training images
        target_size=(100, 100),  # All images will be resized to 150x150
        batch_size=109,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 1098 images belonging to 10 classes.


In [81]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=10,  
      epochs=5,
      verbose=1)

Epoch 1/5
10/10 [==============================] - 10s 1s/step - loss: 0.0548 - acc: 0.9889
Epoch 2/5
10/10 [==============================] - 10s 1s/step - loss: 0.0111 - acc: 0.9970
Epoch 3/5
10/10 [==============================] - 11s 1s/step - loss: 0.0034 - acc: 1.0000
Epoch 4/5
10/10 [==============================] - 10s 977ms/step - loss: 0.0035 - acc: 1.0000
Epoch 5/5
10/10 [==============================] - 11s 1s/step - loss: 0.2404 - acc: 0.9191


In [88]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(100, 100))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  print(np.argmax(classes[0]))
  #if classes[0]>0.5:
  #  print(fn + " is ...")
  #else:
  #  print(fn + " is ....")

Saving 024_1_0003.png to 024_1_0003.png
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
2
